# Introduction to Polars using Python

This workshop is part of the [CDVS Spring 2025 workshop series](library.duke.edu/data/workshops). If you have questions about the content or would like to schedule a consulation with Duke Libraries [Center for Data and Visualization Sciences](library.duke.edu/data), please contact the instructor: Joel Herndon (joel.herndon@duke.edu).

**Abstract**  
Polars provides an open source library for data manipulation that allows fast, large-scale data work in multiple data analytics languages without the need for specialized computing. This workshop focuses on using the Polars library in the Python programming language. We will spend workshop time covering the context for Polars, common data manipulation processes, and close by considering possibilities of using Polars as part of a larger data workflow. This event is open to non-Duke participants.

## Why Polars?

Why another data manipulation language?

1. Speed (especially on laptops/personal machines)
2. Strictness/Consistency
3. Expressiveness

## Workshop goals and guidelines

- This workshop is designed to make you comfortable with Polars fundamentals. The goal of the workshop is to provide enough detail to enable you to evaluate whether Polars is a good fit for your work.
  - We will use fairly simple examples, but Polars is capable of much more complex work.
  - We won't cover all the features available in Polars in this hour long workshop, but I hope you continue to explore!
  - We won't consider all aspects of the Polars API, but I list some resources at the end that will provide details.

- We are using Google's Colab in the live workshop, but I'd encourage everyone to consider a full featured code development environment if you wish to use polars further.

- You are encouraged to ask questions, both during the workshop. We're also happy to stay in touch at askdata@duke.edu after the workshop!

## Core data structures

Polars has two prominent data structures.

1. Series
2. DataFrames

### 1. Series

- named, one-dimensional data structures
- containing sequential data
- all data are of the same type (homogenous)

 Key aspects of a polars Series:

- It is immutable (operations create new Series rather than modifying existing ones)
- It is designed for fast vectorized operations
- It can handle null (missing) values while maintaining strong typing



Let's build two small polars Series comprised of:

1. the names of coffee drinks
2. the prices of coffee drinks

In [12]:
# 1. coffee drink orders
import polars as pl
coffee_drink_orders = pl.Series('coffee_drinks', ['brewed coffee', 'latte', 'mocha', 'espresso', 'latte', 'mocha'])
coffee_drink_orders

coffee_drinks
str
"""brewed coffee"""
"""latte"""
"""mocha"""
"""espresso"""
"""latte"""
"""mocha"""


In [13]:
# 2. prices of coffee drinks
import polars as pl
coffee_drink_prices = pl.Series('coffee_prices', [2.25, 4.25, 5.50, 2.25,4.25, 5.50])
coffee_drink_prices

coffee_prices
f64
2.25
4.25
5.5
2.25
4.25
5.5


### 2. DataFrames

- named, two-dimensional data structures
- containing a collection of Series (columns)
- each column (polars Series) can hold a different data type
- each column (polars Series) must contain the same number of elements

Key aspects of a polars DataFrame:

- It is heterogeneous (different columns can have different data types)
- Columns are named
- It is immutable (operations create new DataFrames rather than modifying existing ones)
- The columns are internally stored as contiguous arrays, optimized for both row and column operations
- It supports vectorized operations across multiple columns

Since we have already have two polars Series (coffee drinks and coffee prices) with information about the coffee names and coffee prices, we can build a polars DataFrame from the two Series that we just created.

In [9]:
# DataFrame of names and prices of coffee drinks
coffee_df = pl.DataFrame(
    {
        'name' : coffee_drink_orders,
        'price': coffee_drink_prices
    }
)
coffee_df

name,price
str,f64
"""brewed coffee""",2.25
"""latte""",4.25
"""mocha""",5.5
"""espresso""",2.25
"""latte""",4.25
"""mocha""",5.5


Another popular approach for creating DataFrame in code is to utilize a group of python dictionaries create the Series that comprise the polars DataFrame.

The following code combines four dictionaries into a new DataFrame called `flights`.

I created this small dataframe of various flights from Raleigh Durham airport (RDU) so that we can explore Polars data wrangling in a way where you can see the transformations clearly. We'll use this dataset in some common data manipulation tasks that follow.

Note: the distance variables is courtesy of: [https://www.airportdistancecalculator.com/](https://www.airportdistancecalculator.com/)

In [ ]:
import polars as pl
flights = pl.DataFrame(
    {
        "flight": ["Raleigh Durham to Svalbard",
                   "Raleigh Durham to London",
                   "Raleigh Durham to Los Angeles",
                   "Raleigh Durham to Boston",
                   "Raleigh Durham to Chicago",
                   "Raleigh Durham to New York"],
        "cost": [None,1200.00,600.00,250.00,300.00, 420.00],
        "distance_km": [6209,6216,3595,985,1040,693],
        "non_stop": [False,True,True, True, False, True],
    }
)
flights

## Common Polars Data Types

Polars has a wide range of [data types](https://docs.pola.rs/api/python/stable/reference/datatypes.html).

For today's workshop, we will focus on:

- Numeric types (on my mac the default is 64 bit floats (decimals) and 64 bit integers)
- Strings
- Booleans

If you look at the output of the `flights` DataFrame above, you can see that Polars will assign default types to the data as they are assigned.


## Loading and Saving Data

Generally, most of the data that you are going to use in Polars won't be created inside your program.

Polars has a broad set of tools for loading and parsing data stored in a variety of common formats.

##### Delimited Files

Polars' `read_csv()` function will read comma delimited files when you provide a location for the file. Polars also uses the `read_csv()` function to load other delimited formats when specify the delimeter using the `separator` argument (see example below).

Let's give it a try.

When you execute the code block below, click the browse button below the code, and upload the `flights.csv` file in the data directory that you downloaded earlier in the class.

In [ ]:
# if you are running this outside of google colab
# omit the first two lines
from google.colab import files
files.upload()
df = pl.read_csv("flights.csv") # basic example
df

In the example above, we had a relatively simple polars DataFrame with clean data. Sometimes, your data may have certain content that requires additional specification to load the data cleanly. Some common examples follow:

1. Your data have missing values encoded as "NA" (a string), but should be treated as missing data.  
`df = read_csv("flights.csv", null_values="NA")`

2. Your data are delimited, but instead of using a comma, use a tab (\t) or other delimiter.  
`df = read_csv("flights.txt", separator="\t") # tab delimited code`

3. Your data contain dates and/or dates with times and you would like for Polars to attempt to convert those fields to the appropriate date or time type.  
`df = read_csv("flights.csv", try_parse_dates=True)`
```

##### Excel

No matter how you feel about Microsoft Excel, it is a very popular format. You can directly load a single sheet of excel data using Polars `read_excel` command.
    
**Note:** Polars uses a helper package to read excel files. In the example that follows, Google Colab appears to require the `fastexcel` package to be installed for this example to work. Please uncomment the first line to install `fastexcel` if needed. If you are using another development environment, please install the necessary helper package. See the [Polars API notes](https://docs.pola.rs/api/python/stable/reference/api/polars.read_excel.html).

When you execute the code block, click browse, and upload the `flights.xlsx` file in the data directory that you downloaded earlier in the class.

In [ ]:
#!pip install fastexcel
from google.colab import files
files.upload()
excel_flights = pl.read_excel("flights.xlsx") #basic example
excel_flights

### Saving Data

In general most of the functions for loading data have a corresponding function for writing data as well. Please see the [Polars API](https://docs.pola.rs/api/python/stable/reference/io.html) for further details.

In [ ]:
df.write_csv("flights.csv") #basic example

### Missing data in Polars

Polars represents missing values using `null` exclusively across data types. When you are loading data, remember that it's important to ensure that other encodings for missing data are converted so that they are correctly represented as missing (and not as strings ("NA"), negative numbers (-99), or some other encoding).

# Common Data Manipulation Processes

One of Polars great strengths as a data manipulation library is its simple, expressive syntax that allows you to construct complex expressions by combining a simple elements. We will discuss Polars expressions shortly, but let's try some hands-on data data wrangling first.

### 5 common "verbs" for data wrangling in Polars

Inspired by a common way of sharing the common commands for data wrangling in the R data analysis langauge, I present five common methods for data wrangling using Polars.


#### 1. select()

The `select` command is the primary method for choosing columns (or variables) that you wish to include in a dataframe. Using our `flights` DataFrame, let's look at a few examples.

In [ ]:
# First, let's remind ourselves of the columns that are available.
# I can do this several ways. Try: `flights.columns` for example.
# Let's look at the entire DataFrame
flights

If I want to see the names of the flights, I can select the `flight` column using the `pl.col` function which tells polars which columns that we wish to select. We will use `pl.col` to select columns in all of these examples.

In [ ]:
flights.select(pl.col('flight'))

If I wanted to see the `flight` and the `distance_km` columns only, I can select those to appear.

In [ ]:
flights.select(pl.col('flight','distance_km'))

You are welcome to select as many columns as you wish, but when you have a lot of columns and you want almost all of them, polars offers the `drop()` method to remove columns that you wish to omit. Let's drop the airport abbreviations in the `origin` and `destination` columns. Typing one variable is a lot faster than typing three!

In [ ]:
flights
flights.drop(pl.col('cost')) # note that I am not saving this change into a new DataFrame

`select()` is a powerful tool. By reducing the number of columns in your dataframe, you increase the efficiency of your analysis (and increase the speed).
  
Note: Polars offers some additional options for selecting columns.

- It is possible to use regular expressions to define a selection `flights.select(pl.col('^c.*$'))` # columns starting with "c"
- It is possible to select by data type
- It is possible to use slice notation and avoid select entirely (but this is not preferred syntax) `flights[:]`

#### 2. filter()
The `filter` command is the primary method for selecting rows based on certain criteria that you wish to include in a dataframe.

Polars filters rows using a column (or vector) of true/false values to determine which rows to keep and which rows to drop (technically this is called "boolean mask filtering").

Using our flights DataFrame, let's look at a few examples.

Let's say that we wanted only the flights from RDU that were of a distance less than 1000 kilometers. We could reduce our DataFrame to these flights with the following code.

In [ ]:
flights.filter(pl.col('distance_km') < 1000)

So, our Boston and New York flights are both under 1000 kilometers from Durham.  

Let's make this a bit more complex, by adding more criteria to the filter.

Polars allows complex filter queries by combining multiple conditions using boolean expressions. For most operations, you will use the following operators to build these expressions.

|Operator | Symbol |
|----------|:----------:|
| AND   | &   |
| OR    | \|   |
| NOT   | ~    |



Polars tends to require explicit (with a few shortcuts- see below) conditions- so expect to type in each condition. Also, Polars wants you to enclose each condition in parentheses. It's really helpful to use a development environment that provides visual cues (usually colored parentheses) or code tips to ensure you've closed the relevant parenthesis.


Let's say that we want to see all the flights that are:

1. less than 6000 kilometers from Durham
2. less than $500

In [ ]:
flights.filter(
    (pl.col('distance_km')<6000) &
    (pl.col('cost')< 500.00)
)

When a column contains a boolean data type (`true` or `false`), you can simply add it into the set of conditions (if the item is `true`, include that row). For example:  

Which flights in our dataset are non-stop from Raleigh Durham?  I know that I have non-stop column that has boolean values (they are `true` or `false` depending on whether the flight is non-stop).?

In [ ]:
flights.filter(pl.col('non_stop'))

Since the column is already contains boolean values, simply listing the column as a filter condition selects the rows where that condition evaluates to `True`.  


**BONUS**: What if you wanted to see the flights with multiple stops?    


Not a problem!

Just use `NOT` boolean operator `~` like so:

In [ ]:
flights.filter(~pl.col('non_stop'))

#### 3. sort()

In the previous exercises, `select` and `filter` have helped to reduce our Dataframe to relevant columns and rows.

The `sort` method builds on these commands by allowing us to specify how the columns should be arranged for presentation.

Let's try a simple sort based on our filtered data considering non-stop flights in the dataset.



In [ ]:
flights.filter(pl.col('non_stop')).sort(pl.col('distance_km'))

In the above, note how `sort` defaults to ascending order. If you want to sort in descending order:

In [ ]:
flights.filter(pl.col('non_stop')).sort(pl.col('distance_km'), descending=True)

It is entirely possible to specify more than one column for a sort. Let's look at our non-stop flights that have multiple stops and sort by the cost and the distance_km.

In [ ]:
flights.filter(pl.col('non_stop')).sort([
    pl.col('cost'),
    pl.col('distance_km')])

*Optional*: Is it possible to sort some columns ascending with others descending? Yes, just assign bracketed list of booleans (`True` or `False`) to each of the sort columns.

In [ ]:
flights.filter(pl.col('non_stop')).sort([
    pl.col('cost'),
    pl.col('distance_km')],
    descending=[True, False])

Note how our data are now sorted by the distance in kilometers in ascending order. Often, we will want to see the results in descending order (top 5 lists are very popular in data work!). It is easy to specify the `descending` parameter in the `sort` method as `True` to do this.

One last thing to not about sorting DataFrames.  If you have missing data in a sort column, it will be listed first (it doesn't matter if you are sorting ascending or descending). If you would prefer to have the `null` values appear at the end of the sort, add the `nulls_last=True` parameter.

Nulls first:

In [ ]:
flights.sort(pl.col('cost'), descending=True)

Nulls last:

In [ ]:
flights.sort(pl.col('cost'), descending=True, nulls_last=True)

It is entirely possible to specify more than one column for a `sort`. Let's look at our flights that have multiple stops (non-stop = False) and sort by the `cost` and the `distance_km`.

#### 4. with_columns()

We often want to build on the data that we have in our dataset by adding addtional variables to our datasets. The `with_columns()` method provides a convenient way to Adding columns to DataFrames.  Let's calculate the cost per kilometer for each of flights and try to determine which is the best bargain.



In [ ]:
flights.with_columns(
   cost_per_kilometer = pl.col('cost') / pl.col('distance_km')
)

The flight to Los Angeles is a slightly better deal in the cost per kilometer than the flight to London. Note that the we didn't have a cost for the flight to Svalbard (we only have a `null` value there) and we couldn't calculate the cost per kilometer for that trip.

Up until now, we've been selecting, filtering, and sorting our data without making any changes in the underlying DataFrame. As we start adding new variables (or new columns) like the "cost per kilometer", we will likely want to save our new column. In order to do this, you must assign the result of the `.with_columns()` expression to a new (or existing) variable name like so:

In [ ]:
flights = flights.with_columns(
   cost_per_kilometer = pl.col('cost') / pl.col('distance_km')
)
flights

Another stylistic note- Polars offers two different syntax styles for creating columns.

1. *keyword argument syntax*: In the example above, we name the column, apply the `=` sign and define our calculation.  

2. *expression based syntax*: Polars also has an expression based syntax that assigns the name of the variable by using an `.alias()` method at the end of the expression. This may seem more intuitive if you use python a lot and enjoy method chaining. (If that doesn't make sense- stick with the keyword argument syntax!)

```
flights = flights.with_columns(
   (pl.col('cost') / pl.col('distance_km')).alias('cost_per_kilometer')
)
flights
```

#### 5. group_by()


Sometimes you want to aggregate your data to understand patterns by groups. The `group_by()` method provides a convenient way to collapse the dataframe into groups defined by one or more variables and then generate statistics based on these groupings. Note that `group_by()` on it's own only splits the data into groups (see code below) without producing results, we have to add some more code to run calculations on the groups.


In [ ]:
flights.group_by(pl.col('non-stop')) # this creates groups... but no output!

For now, we will start with a very simple example using our small flights dataset that considers the average distance of non-stop flights versus flights with layovers. The code after `.agg()` calculates the mean of the distance in kilometers and renames the calculated variable `average_distance_km`.

In [ ]:
flights.group_by(pl.col('non_stop')).agg(pl.col('distance_km').mean().alias('average_distance_km'))

#### 6. over()

I know. I said I was only going to show you five common "verbs" in Polars...

 Some of you are probably looking at the `group_by()` function and thinking "that's really great for aggregating the data and creating summaries by categories.... BUT what if I want to aggregate the data by groups and then add that calculation to each row that is a member of that group?  This is where the `over()` method comes in handy in Polars. Let's try the same example as above, but this time we'll use `.over` in our expression instead of `group_by()`.

 In the output that follows, you can see how average distance is calculated for each group in the non-stop variable (`true` and `false`) and then added to the Dataframe as a new column. We use polars `.alias` method to provide a name for the newly calculated column.

In [ ]:
flights.with_columns(pl.col('distance_km').mean().over(pl.col('non_stop')).alias('average_distance_km'))

### Expressions

Let us focus on a core feature of polars: **expressions**. We've been using expressions already in our small data examples without noting it. As Jannsens and Nieuwdorp (2025) describe it:

> "If you think of an expression as a recipe, then the operations would be the steps, and the functions and methods would be like the cooks."

Jannsens and Nieuwdorp (2025) also provide a formal definition:

> "An expression is a tree of operations that describe how to constuct one or more Series."

Let's look at an example to make this clearer. In the code below, we use the DataFrame method `with_columns` to add a column to the DataFrame. In this example, you can think of of `with_columns` as the chef and the expression that follows as the steps to make it happen. Indeed, the steps or the expression is completely reusable in a variety of contexts. Let's try it!

In [ ]:
# run this code for reminder of how with_columns works
# we've seen this earlier in the workshop
flights.with_columns(
   cost_per_kilometer = pl.col('cost') / pl.col('distance_km')
)

In [ ]:
# now, let's extract the expression into a variable
my_expression = cost_per_kilometer = pl.col('cost') / pl.col('distance_km')

My instructions for creating a `cost_per_kilometer` are now saved in the variable `my_expression`. This is valid code, but without the DataFrame and a method ("verb") to activate the code, if I call the expression I just see the code.

In [ ]:
# Note that the expression is valid code (you won't get an error)
# BUT also note that you need a DataFrame and one of our methods ("verbs") that
# acts on the expression to generate a result.
my_expression

Let's use our expression in a different context. I'm still working with the same `flights` data that we used before, but this time I'm going to use the `select` method of the DataFrame. Note that you can use `select` to generate Series in a DataFrame as well as "selecting" Series in DataFrame.

In [ ]:
# and now, I'm going to reuse my_expression with the same Dataframe
# but with a different verb (or method)
flights.select(
    my_expression
)

If this seems confusing, don't worry! Expressions tend to make more sense as you use Polars more frequently.

## Polars Workflows

### Big data and Polars

The six observations in the flights dataset provide a quick visual idea of polars data manipulation tools, but do not illustrate polars ability to process large data collections. For this section of the workshop, we will scale up our current flights dataset to examine air travel data to Raleigh Durham airport from 2010 to 2024 using data from the Federal Aviation Administration (FAA). A special thanks to Simon Couch and his colleagues who developed the [R package "anyflights"](https://github.com/simonpcouch/anyflights) that provided the tools for downloading the FAA data!

Let us load these data into Colab and explore Polars with a dataset of roughly a million rows. When you execute the code that follows, click the browse button that appears, and upload the `rdu_flights.parquet` file in the data directory that you downloaded earlier in the class. Note that we are using the [parquet format](https://en.wikipedia.org/wiki/Apache_Parquet) which stores very large files efficiently. When the file loads, you will note that we get a report of the first five rows and the last five rows of the DataFrame. Polars is designed for working for large data- so the default response takes the size of the data into account.

In [4]:
# if you are running this outside of google colab
# omit the first two lines
import polars as pl #if neeeded
from google.colab import files
files.upload() # choose rdu_flights.parquet
rdu_flights = pl.read_parquet("rdu_flights.parquet") # basic example
rdu_flights

Saving rdu_flights.parquet to rdu_flights.parquet


year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier_code,flight,tailnum,origin,dest,air_time,distance,hour,minute,departure_date_time,carrier_name,temp,dewp,humid,wind_dir,wind_speed,wind_gust,precip,pressure,visib,time_hour.y,destination_airport_names,destination_airport_lat,destination_airport_lon,destination_airport_alt,destination_airport_tz,destination_airport_dst,destination_airport_tzone,plane_year,plane_type,plane_manufacturer,plane_model,plane_engines,plane_seats,plane_speed,plane_engine
i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,str,str,str,i64,i64,i64,i64,"datetime[μs, America/New_York]",str,f64,f64,f64,i64,f64,f64,f64,f64,f64,"datetime[μs, UTC]",str,f64,f64,i64,i64,str,str,i64,str,str,str,i64,i64,i64,str
2010,1,1,553,600,-7,709,728,-19,"""FL""",12,"""N934AT""","""RDU""","""ATL""",62,356,6,0,2010-01-01 06:00:00 EST,"""AirTran Airways Corporation""",42.8,42.8,100.0,240,3.45234,3.972884,0.0,null,1.5,2010-01-01 11:00:00 UTC,"""Hartsfield Jackson Atlanta Int…",33.6367,-84.428101,1026,-5,"""A""","""America/New_York""",2000,"""Fixed wing multi engine""","""BOEING""","""717-200""",2,100,0,"""Turbo-fan"""
2010,1,1,555,600,-5,720,740,-20,"""MQ""",269,"""N807MQ""","""RDU""","""JFK""",64,426,6,0,2010-01-01 06:00:00 EST,"""Envoy Air""",42.8,42.8,100.0,240,3.45234,3.972884,0.0,null,1.5,2010-01-01 11:00:00 UTC,"""John F Kennedy International A…",40.639801,-73.7789,13,-5,"""A""","""America/New_York""",null,null,null,null,null,null,null,null
2010,1,1,609,610,-1,755,813,-18,"""DL""",181,"""N368NB""","""RDU""","""MSP""",145,981,6,10,2010-01-01 06:00:00 EST,"""Delta Air Lines Inc.""",42.8,42.8,100.0,240,3.45234,3.972884,0.0,null,1.5,2010-01-01 11:00:00 UTC,"""Minneapolis-St Paul Internatio…",44.882,-93.221802,841,-6,"""A""","""America/Chicago""",2003,"""Fixed wing multi engine""","""AIRBUS""","""A319-114""",2,145,0,"""Turbo-fan"""
2010,1,1,613,615,-2,825,830,-5,"""AA""",170,"""N4WAAA""","""RDU""","""DFW""",172,1062,6,15,2010-01-01 06:00:00 EST,"""American Airlines Inc.""",42.8,42.8,100.0,240,3.45234,3.972884,0.0,null,1.5,2010-01-01 11:00:00 UTC,"""Dallas Fort Worth Internationa…",32.896801,-97.038002,607,-6,"""A""","""America/Chicago""",null,null,null,null,null,null,null,null
2010,1,1,617,618,-1,714,726,-12,"""UA""",58,"""N494UA""","""RDU""","""IAD""",38,224,6,18,2010-01-01 06:00:00 EST,"""United Air Lines Inc.""",42.8,42.8,100.0,240,3.45234,3.972884,0.0,null,1.5,2010-01-01 11:00:00 UTC,"""Washington Dulles Internationa…",38.9445,-77.455803,312,-5,"""A""","""America/New_York""",2002,"""Fixed wing multi engine""","""AIRBUS""","""A320-232""",2,200,0,"""Turbo-fan"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024,9,30,2117,2051,26,2206,2155,11,"""AA""",111,"""N131NN""","""RDU""","""CLT""",27,130,20,51,2024-09-30 20:00:00 EDT,"""American Airlines Inc.""",null,null,null,null,null,null,null,null,null,null,"""Charlotte Douglas Internationa…",35.214001,-80.9431,748,-5,"""A""","""America/New_York""",2015,"""Fixed wing multi engine""","""AIRBUS""","""A321-231""",2,379,0,"""Turbo-fan"""
2024,9,30,2203,2127,36,2332,2259,33,"""F9""",188,"""N338FR""","""RDU""","""LGA""",64,431,21,27,2024-09-30 21:00:00 EDT,"""Frontier Airlines Inc.""",null,null,null,null,null,null,null,null,null,null,"""La Guardia Airport""",40.777199,-73.872597,21,-5,"""A""","""America/New_York""",2018,"""Fixed wing multi engine""","""AIRBUS""","""A320-251N""",2,190,0,"""Turbo-jet"""
2024,9,30,2203,2204,-1,7,4,3,"""F9""",140,"""N354FR""","""RDU""","""DFW""",141,1061,22,4,2024-09-30 22:00:00 EDT,"""Frontier Airlines Inc.""",null,null,null,null,null,null,null,null,null,null,"""Dallas Fort Worth Internationa…",32.896801,-97.038002,607,-6,"""A""","""America/Chicago""",2019,"""Fixed wing multi engine""","""AIRBUS""","""A320-251N""",2,190,0,"""Turbo-jet"""


### Getting a sense of the data
Before we start exploring the `rdu_flights` data, let us first try to understand a little bit about how this data differs from our six flight example that we used earlier. If you look at the top of the of the DataFrame listing above, you will note that Polars already provides information about the DataFrame each time you print it. The shape (954_988, 45) tells us that we have 954988 rows (the underscore is python's way of making the number easier to read) and 45 columns.  It is possible to ask polars directly for the number of rows and columns by using the `height` and `width` properties of the DataFrame at any time. We can also ask for the column names (`columns`) to get a sense of the variables.

Note: If you are using polars heavily for data exploration, I'd strongly recommend using an intergrated development environment (IDE) for coding and data analysis. Several IDEs have powerful and convenient tools for exploring data and variables that will allow you to focus on analytics.

In [ ]:
rdu_flights.height # how many rows in the dataframe?

In [ ]:
rdu_flights.width # how many columns?

45

In [ ]:
rdu_flights.columns # what are the column names?

In [ ]:
rdu_flights.schema # let's look at the column names and data types together

In [5]:
rdu_flights.describe() # descriptive statistics for the dataframe (you have to look at the dep_delay column)

statistic,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier_code,flight,tailnum,origin,dest,air_time,distance,hour,minute,departure_date_time,carrier_name,temp,dewp,humid,wind_dir,wind_speed,wind_gust,precip,pressure,visib,time_hour.y,destination_airport_names,destination_airport_lat,destination_airport_lon,destination_airport_alt,destination_airport_tz,destination_airport_dst,destination_airport_tzone,plane_year,plane_type,plane_manufacturer,plane_model,plane_engines,plane_seats,plane_speed,plane_engine
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,str,str,str,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,str,str,f64,str,str,str,f64,f64,f64,str
"""count""",954988.0,954988.0,954988.0,933410.0,954988.0,933372.0,932515.0,954988.0,930678.0,"""954988""",954988.0,"""949543""","""954988""","""954988""",930702.0,954988.0,954988.0,954988.0,"""954988""","""954988""",298757.0,298698.0,298697.0,876682.0,903195.0,903195.0,244561.0,263890.0,922862.0,"""923036""","""952641""",952641.0,952641.0,952641.0,952641.0,"""952641""","""952641""",837604.0,"""857448""","""857448""","""857448""",857429.0,857426.0,857448.0,"""857448"""
"""null_count""",0.0,0.0,0.0,21578.0,0.0,21616.0,22473.0,0.0,24310.0,"""0""",0.0,"""5445""","""0""","""0""",24286.0,0.0,0.0,0.0,"""0""","""0""",656231.0,656290.0,656291.0,78306.0,51793.0,51793.0,710427.0,691098.0,32126.0,"""31952""","""2347""",2347.0,2347.0,2347.0,2347.0,"""2347""","""2347""",117384.0,"""97540""","""97540""","""97540""",97559.0,97562.0,97540.0,"""97540"""
"""mean""",2017.423435,6.491942,15.745004,1258.682642,1250.621039,10.132992,1402.463886,1408.937122,4.868246,null,152.088545,null,null,null,95.289715,626.541707,12.24622,25.999054,"""2017-12-02 22:11:06.456960-05:…",null,59.463573,47.360029,67.698831,136.641775,5.74714,6.613693,0.005341,1018.147586,9.270627,"""2017-12-03 02:02:58.557716+00:…",null,37.02327,-84.298705,558.739148,-5.442069,null,null,2004.660361,null,null,null,1.993812,132.165919,1.150801,null
"""std""",3.816657,3.401967,8.766673,484.650805,469.664462,48.808198,491.690245,474.995666,50.915639,null,105.25501,null,null,null,63.339855,481.898839,4.672328,18.282786,null,null,17.781744,18.282563,20.663515,115.293381,4.427147,5.094672,0.028061,6.677732,2.009998,null,null,5.033743,11.509966,971.185879,0.77921,null,null,19.880725,null,null,null,0.095446,62.354788,17.679796,null
"""min""",2010.0,1.0,1.0,1.0,45.0,-67.0,1.0,1.0,-78.0,"""9E""",1.0,"""(null)""","""RDU""","""ALB""",11.0,113.0,0.0,0.0,"""2010-01-01 06:00:00-05:00""","""AirTran Airways Corporation""",7.0,-11.0,8.51,0.0,0.0,0.0,0.0,990.8,0.0,"""2010-01-01 11:00:00+00:00""","""Albany International Airport""",25.7932,-122.375,4.0,-8.0,"""A""","""America/Chicago""",0.0,"""Balloon""","""AERO COMMANDER""","""150""",1.0,1.0,0.0,"""2 Cycle"""
"""25%""",2015.0,4.0,8.0,812.0,807.0,-5.0,947.0,953.0,-15.0,null,68.0,null,null,null,58.0,356.0,8.0,10.0,"""2015-06-24 06:00:00-04:00""",null,45.0,33.0,50.94,10.0,2.30156,2.648589,0.0,1013.9,10.0,"""2015-06-24 15:00:00+00:00""",null,33.6367,-87.9048,21.0,-6.0,null,null,2000.0,null,null,null,2.0,88.0,0.0,null
"""50%""",2018.0,7.0,16.0,1236.0,1230.0,-2.0,1400.0,1404.0,-7.0,null,134.0,null,null,null,74.0,442.0,12.0,25.0,"""2018-05-19 07:00:00-04:00""",null,61.0,50.0,70.64,140.0,5.7539,6.621473,0.0,1017.9,10.0,"""2018-05-19 10:00:00+00:00""",null,38.9445,-81.308998,146.0,-5.0,null,null,2005.0,null,null,null,2.0,140.0,0.0,null
"""75%""",2020.0,9.0,23.0,1714.0,1705.0,5.0,1843.0,1845.0,7.0,null,219.0,null,null,null,103.0,667.0,17.0,41.0,"""2020-04-05 09:00:00-04:00""",null,73.0,64.0,86.66,230.0,8.05546,9.270062,0.0,1022.4,10.0,"""2020-04-05 16:00:00+00:00""",null,40.777199,-75.241096,748.0,-5.0,null,null,2011.0,null,null,null,2.0,179.0,0.0,null
"""max""",2024.0,12.0,31.0,2400.0,2355.0,2826.0,2400.0,2359.0,2815.0,"""YX""",610.0,"""N9EAMQ""","""RDU""","""XNA""",626.0,2400.0,23.0,59.0,"""2024-09-30 22:00:00-04:00""","""Virgin America""",10

The `rdu_flights` DataFrame list all the flights with Raleigh Durhgam airport as the `origin`. The DataFrame also has rich information about the nature of the flight (departure time versus scheduled departure for example), the weather at RDU at the time of the flight, and the type of plane used for the flight. For todays workshop, we only have time to explore a bit, but you are welcome to use this dataset to explore (and maybe form some hypothesis about?) flights out of Raleigh Durham airport.

### Exploratory Analysis: Example 1

The primary reason that most people consider polars is its speed at processing large data collections. Let's ask some questions about our data and note the speed with roughly a million observations.

1. What are the number of departing flights at RDU airport by year?

We can't directly see a million observations, but we might be interested in the number of flights that originate from Raleigh Durham airport each year from 2010 to 2024 (we only have data through September 2024). Let's group the DataFrame by year and create an expression to count the flights (each row in this DataFrame is a flight) using the `len()` method.

In [ ]:
rdu_flights.group_by('year').agg(pl.len())

year,len
i64,u32
2014,38813
2015,34898
2021,44136
2018,121448
2022,54049
2011,42620
2023,59416
2010,48924
2013,47983


This is great... the polars algorithm is fast (instantaneous), but it doesn't return the data sorted in a way that makes it easy to parse. We'd like to have those years sorted.

I'll also note that polars default way of printing DataFrames is great for looking at big data, but very frustrating for smaller DataFrames, we'll change the default.  If you use this code: `pl.Config.set_tbl_rows(100)`, Polars will display up to 100 rows without breaks.

In [ ]:
pl.Config.set_tbl_rows(100) # show 100 rows
rdu_flights.group_by('year').agg(pl.len()).sort('year')

This is great, but we might want to share this with someone. Let us rename the second column from `len` to `flights` How would you do that?

In [ ]:
rdu_flights.group_by(pl.col('year')).agg(pl.len().alias('flights')).sort(pl.col('year'))

This is very handy. I can see the flights increasing at RDU up to the pandemic, and a steady recovery in the last few years (remember, we still don't have a full year of data for 2024). This calculation is so handy, that I'm going to save this expression `pl.len.alias('flights')` so I can use it later without having to retype the entire expression.

In [ ]:
number_of_flights = (pl.len().alias('flights'))

### Exploratory Analysis: Example 2

2. Are departure delays increasing or decreasing at RDU each year?

We have a variable called `dep_delay` that measures delays in minutes (I note that some flights seem to leave early because there are negative delays). Let's count the number of rows each year where the departure delay is greater than 15 minutes.

In [ ]:
rdu_flights.group_by('year').agg(
    pl.col('dep_delay')
    .filter(pl.col('dep_delay') > 15)
    .len()
    .alias('delayed_flights')
).sort('year')

If I'm a researcher for the FAA (or RDU) interested in flight delay data, I might want to use that expression again later. Let's save it in a variable called "flights_delayed_by_15_minutes".


In [ ]:
flights_delayed_by_15_minutes = (
    pl.col('dep_delay')
    .filter(pl.col('dep_delay') > 15)
    .len()
    .alias('delayed_flights')
)


Now let's used our saved expression (`flights_delayed_by_15_minutes`) in the aggregation (`.agg()`) part of our grouping.

In [ ]:
rdu_flights.group_by('year').agg(
    flights_delayed_by_15_minutes
).sort('year')

### Exploratory Analysis: Example 3

3. Is there some relationship with increases in the number of flights and delays?

We've already calculated the number of flights per year and saved that expression (`number_of_flights`).

We've also saved the expression that calculates the flights delayed by fifteen minutes (`flights_delayed_by_15_minutes`).

We can run the same analysis that groups the data by year and simply provide those two expressions to see our flights and delays together.

The `.agg` function will accept a python list (`[]`) of expressions and run each calculation for us.

Let's give it a try:

In [ ]:
rdu_flights.group_by('year').agg([
    number_of_flights,
    flights_delayed_by_15_minutes]
).sort('year')

### Exploratory Analysis: Example 4 (if we have time)

4. What's the best day of the week in 2014 to fly to Washington, DC (Ronald Reagan National Airport or DCA) if you want the best chance of departing on time?

Translation to Polars

1. We want flights in 2014: `filter(pl.col('year') = 2014)`
2. We want flights to DCA so: `filter(pl.col('dest') == "DCA")`
3. We also want flights that depart on time so: `filter(pl.col('dep_delay') <= 0)`

As we learned earlier, polars allows us to combine these conditions in a single filter command. If the filters are additive (each condition is linked by an AND), polars will allow to list the conditions separated by commas. It will know that all the conditions must apply.


4. We haven't covered dates in polars yet, but we have a variable called `departure_date_time` that is a polars `datetime` type. This allows us to query that field for the numerical day of the week (Monday is 1 and Sunday is 7) using this code: `pl.col('departure_date_time).dt.weekday()`. We'll add this to our DataFrame using `with_columns()`.

5. In order to calculate the best chance of of departing on time (on any airline) to Reagan National, we need to calculate the average number of on time (or early departures) on average for each day of the week. This calculation requires two groupings.
  1. Our first `group_by` creates a DataFrame with the day of the week, the date and the number of flights.
  2. The second `group_by` groups the DataFrame from the previous step by day and calculates the average of on-time flights for that day.

Let's put these together in code and see what we get.


In [ ]:
# Coding challenge: can you convert the numeric day?
rdu_flights.filter(
    pl.col('year') == 2024,
    pl.col('dest') == "DCA",
    pl.col('dep_delay') <= 0
).with_columns(
    pl.col('departure_date_time').dt.weekday().alias('day_of_week')
).group_by([pl.col('day_of_week'), pl.col('departure_date_time').dt.date()]).agg(
    pl.len().alias('daily_flights')
).group_by('day_of_week').agg(
    pl.col('daily_flights').mean().alias('avg_daily_on_time_flights')
).sort('avg_daily_on_time_flights', descending = True)

day_of_week,avg_daily_on_time_flights
i8,f64
2,6.128205
7,5.871795
1,5.75
3,5.435897
4,5.410256
5,5.216216
6,3.157895


Based on 2024 flights, your best bet is flying on Tuesday!

## Polars Lazy (and extremely fast) API

Up to this point, we have used Polars eager application programming interface (API) to process our queries. When we enter commands, we get immediate feedback. This immediate feedback resembles other data manipulation libraries like Pandas (python) and dplyr (R), and should be familar if you have worked in those languages.

Polars *lazy API* provides an alternative way of processing large data or working in high performance workflows. Instead of loading data and immediately executing transfomations, Polars evaluates the DataFrame noting the characteristics of the data without loading the data and builds a query graph of the transformations requested. By deferring immediate execution and not loading data until called (the "lazy" in Polars "lazy" API), the lazy API can significantly optimize data manipulation queries.

We won't spend a lot of time looking at Lazy API in today's workshop, but we explore some of its core commands here.

### Using the Lazy API

Implementing the lazy API, generally begins with creating a "LazyFrame" (or representation of the DataFrame that does not contain the data). If you have a DataFrame in memory, you can use the `lazy()` method to convert it to a LazyFrame and use the Polars lazy API.

In [ ]:
lazy_rdu_flights = rdu_flights.lazy()

It's also possible to scan data into a Polars LazyFrame using one of Polars many "scan" methods.

In [ ]:
lazy_rdu_flights = pl.scan_parquet("rdu_flights.parquet")

Once you have loaded your data into a lazyframe, the data manipulation steps are identical to the ones that we have used above with one exception - when you start with a LazyFrame, Polars will not provide a DataFrame in the output until you "materialize" the DataFrame using the `collect()`.

In the examples that follow. The first example uses polars eager API (as we did earlier) to evaluate our query. The second example uses the same query with the lazy API.  I've timed each query (I note that I get varying results on Google Colab...), but generally the lazy API is faster. If you have very big data (more than one million rows and many columns), you should definitely consider the lazy API as part of your analytics strategy!

In [ ]:
# Lets use the Polars eager API
%time
rdu_flights.group_by('year').agg([
    number_of_flights,
    flights_delayed_by_15_minutes]
).sort('year')

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.34 µs


year,flights,delayed_flights
i64,u32,u32
2010,48924,7339
2011,42620,6234
2012,44016,6099
2013,47983,8003
2014,38813,6757
…,…,…
2020,67720,5082
2021,44136,5674
2022,54049,9896


In [ ]:
# Lets use the lazy API
%time
lazy_rdu_flights.group_by('year').agg(
    [
        number_of_flights,
        flights_delayed_by_15_minutes
    ]).sort('year').collect()


CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.25 µs


year,flights,delayed_flights
i64,u32,u32
2010,48924,7339
2011,42620,6234
2012,44016,6099
2013,47983,8003
2014,38813,6757
…,…,…
2020,67720,5082
2021,44136,5674
2022,54049,9896


# Closing Thoughts and Questions



## Getting Help

Learning Polars is an iterative process that takes time. It can be frustrating to translate your analysis goals into a new language-- especially if you are very skilled in a given language. The following resources may be of help:  

### General resources

- *CDVS - Duke Libraries - askdata@duke.edu*  
As always, Duke Libraries Center for Data and Visualization Science (askdata@duke.edu) can assist with questions about data management and data wrangling. Consultations are available bny

- *Polars API*  
While I sometimes disagree with how things are organized... the [Polars Python API](https://docs.pola.rs/api/python/stable/reference/index.html) is an outstanding resource for the latest syntax. If you are questioning the validity of AI suggestions (and sometimes those suggestions are erroneous!), the API can help resolve questions.

### LLM ("AI") Resources
- [Claude.ai](claude.ai) is quite good at code related questions (although you should always verify what it tells you!).
- [Microsoft's Co-Pilot](https://oit.duke.edu/service/microsoft-copilot/) is available at Duke and can be helpful.
- [Github for Education](https://github.com/education) (*Students and Educators* by application) - Github offers very generous access to its  tools which include github copilot.

### eBooks
Duke Libraries subscribe to the [O'Reilly for Higher Education](https://go.oreilly.com/duke-university) Database where you will find Jeroen Janssens and Thijs Nieuwdorp's [Python Polars: The Definitive Guide](https://learning.oreilly.com/library/view/python-polars-the/9781098156077/). This is an excellent way to learn Polars and serves as compelling reference. Note that the book will be published (in print!) in April 2025.

### Data resources
We used FAA flight data pulled using the [anyflights]() API. If you would like to investigate this API further check out:  

Couch S (2023). anyflights: Query 'nycflights13'-Like Air Travel Data for Given Years and Airports. https://github.com/simonpcouch/anyflights, https://simonpcouch.github.io/anyflights/.

### Other polars APIs

As mentioned in the introduction, [Polars](https://pola.rs/) is written in Rust and has implementations in Julia, R (check out [TidyPolars](https://tidypolars.etiennebacher.com/), and Javascript. If Polars seems compelling and you regularly use one of those languages, please try one of the other implementations!

